In [ ]:
using Plots
using LaTeXStrings
using GLMakie
using Distributions, Random
using LinearAlgebra
# using ProfileVega
# using Traceur

# Parameters

In [ ]:
set_zero_subnormals(true)

In [ ]:
const global U0 = 10
const global a = 2

const N = 1000
const global xmax = 6
const global xmin = -6
const global ymax = 6
const global ymin = -6
const global G = [0, -9.81];
# FLUID
ρ = 997
μ = 1e-3
const global k_b = 1.3806490e-23

In [ ]:
function Stokes(d)
    return ρ_d * (d^2) / (18 * μ) * U0 / 6
end

function invStokes(St)
    return sqrt(18 * St * μ * 6 / (U0 * ρ_d))
end

# Functions 

In [ ]:
function RungeKutta(fun, t0, tn, dt, y0, checkFun)
    y = []
    push!(y, y0)

    steps = round(Int, (tn - t0) / dt)
    for i in 1:steps
        k1 = dt .* fun(checkFun(y[i]))
        k2 = dt .* fun(checkFun(y[i] .+ k1 ./ 2))
        k3 = dt .* fun(checkFun(y[i] .+ k2 ./ 2))
        k4 = dt .* fun(checkFun(y[i] .+ k3))
        k = (k1 + 2 * k2 + 2 * k3 + k4) / 6
        push!(y, checkFun(y[i] + k))
    end
    return y
end

function RK4systems(f, g, t0, tn, dt, y0, V0, checkFun)
    #    RK4systems(dxdtP, dvdtP, t0, tend, dt, points, V0);
    x = Vector{Matrix{Float64}}(undef, 0)
    v = Vector{Matrix{Float64}}(undef, 0)

    k1 = zeros(size(y0))
    k2 = zeros(size(y0))
    k3 = zeros(size(y0))
    k4 = zeros(size(y0))

    l1 = zeros(size(V0))
    l2 = zeros(size(V0))
    l3 = zeros(size(V0))
    l4 = zeros(size(V0))

    push!(x, y0)
    push!(v, V0)
    steps = round(Int, (tn - t0) / dt)
    for i in 1:steps
        f(k1, x[i], v[i])
        g(l1, x[i], v[i])

        f(k2, x[i] .+ (k1 ./ 2), v[i] .+ l1 ./ 2)
        g(l2, x[i] .+ (k1 ./ 2), v[i] .+ l1 ./ 2)

        f(k3, x[i] .+ (k2 ./ 2), v[i] .+ l2 ./ 2)
        g(l3, x[i] .+ (k2 ./ 2), v[i] .+ l2 ./ 2)

        f(k4, x[i] .+ k3, v[i] .+ l3)
        g(l4, x[i] .+ k3, v[i] .+ l3)

        k = (k1 .+ 2 .* k2 .+ 2 .* k3 .+ k4) ./ 6
        l = (l1 .+ 2 .* l2 .+ 2 .* l3 .+ l4) ./ 6
        push!(x, checkFun(x[i] + k, v[i] + l)[1])

        push!(v, v[i] + l)
        if any(isnan, x[i])
            println("Blew UP at $i")
            break
        end
        println(maximum(x[i+1] - x[i])^2)
    end
    return x
end

function meshgrid(x, y)
    X = [i for i in x, j in 1:length(y)]
    Y = [j for i in 1:length(x), j in y]

    return cat(X, Y, dims=3)
end

function Pressure(coord, vfun)
    U = vfun(points)
    @views u = U[1, :]
    @views v = U[2, :]
    return @. 0.5ρ * U0^2 - 0.5ρ * (u^2 + v^2)
end

function isoPsi(C, tol, zs, xs, ys, N)
    sols = []
    for i in 1:N
        for j in 1:N
            if abs(zs[i, j] - C) < tol
                push!(sols, [xs[i], ys[j]])
            end
        end
    end
    return sols
end

function dxdtP!(DX, points, V)
    DX[:] = dt .* V
    nothing
end

function Knudsen(coord)
    L = R
    p = 0.5ρ * U0^2 * ones(size(coord))
    Kn = @. k_b * T / (sqrt(2) * π * d_p^2 * p * L)
    return Kn
end

function Brownian(coord, V)
    d = Normal(0, 1)
    Gaussian = rand(d, size(coord))

    Kn = Knudsen(coord)
    C_c = @. 1 / (1 + Kn * (2.49 + 0.84 * exp(-1.74 / Kn)))
    S = @. 216 * (μ / ρ) * k_b * T / ((π^2) * rhoDisp * (d_p^5) * (rhoDisp / ρ)^2 * C_c)

    f_t = @. massP * Gaussian * sqrt(π * S / dt)
    return @. (f_t) / massP
end

function periodic(x, V)
    return (mod.(-(6 .- (x)), 12) .- 6), V
end

function periodic(x)
    return (mod.(-(6 .- (x)), 12) .- 6)
end

### Plotting

In [ ]:
function plotTrajcecotries(trajs, timer, fieldContour, colors, markers, tit, divider)
    xs = Vector{Observable{Vector{Float64}}}(undef, size(trajs))
    ys = Vector{Observable{Vector{Float64}}}(undef, size(trajs))
    for i in 1:size(trajs)[1]
        xs[i] = Observable(trajs[i][1][1, :])
        ys[i] = Observable(trajs[i][1][2, :])
    end

    title = Observable(L"%$tit $t = %$(timer[1]) $")

    fig = GLMakie.Figure()
    display(fig)
    ax = Axis(fig[1, 1])

    GLMakie.contourf!(x, y, fieldContour, levels=20, color=:turbo)

    for i in 1:size(trajs)[1]
        GLMakie.scatter!(xs[i], ys[i], color=colors[i], marker=markers[i], strokewidth=2e-4)
    end

    Label(fig[1, 1, Top()], title, padding=(0, 0, 10, 0))
    Colorbar(fig[1, 2]; label=L"Ψ")

    steps = 10e100
    for i in 1:size(trajs)[1]
        if size(trajs[i])[1] < steps
            steps = size(trajs[i])[1]
        end
    end
    framerate = round(Int, steps / divider)
    for i in 1:framerate:steps
        sleep(1 / 30)
        title[] = L"%$tit $t = %$(timer[i]) $"
        for j in 1:size(trajs)[1]
            patching = size(trajs[j][1][1, :])[1] - size(trajs[j][i][1, :])[1]
            xs[j][] = vcat(trajs[j][i][1, :], (2 * xmax) .* ones(patching))
            ys[j][] = vcat(trajs[j][i][2, :], (2 * ymax) .* ones(patching))
        end
    end
end

function saveTrajcecotries(trajs, timer, fieldContour, colors, markers, tit, filename, divider)
    xs = Vector{Observable{Vector{Float64}}}(undef, size(trajs))
    ys = Vector{Observable{Vector{Float64}}}(undef, size(trajs))
    cur_colors = theme_palette(:auto)
    for i in 1:size(trajs)[1]
        xs[i] = Observable(trajs[i][1][1, :])
        ys[i] = Observable(trajs[i][1][2, :])
    end
    title = Observable(L"$tit $t = %$(timer[1]) $")

    fig = GLMakie.Figure()
    display(fig)
    ax = Axis(fig[1, 1])

    GLMakie.contourf!(x, y, fieldContour, levels=20, color=:turbo)

    for i in 1:size(trajs)[1]
        GLMakie.scatter!(xs[i], ys[i], color=colors[i], marker=markers[i], strokewidth=2e-4)
    end

    Label(fig[1, 1, Top()], title, padding=(0, 0, 10, 0))
    Colorbar(fig[1, 2]; label=L"Ψ")

    steps = 10e100
    for i in 1:size(trajs)[1]
        if size(trajs[i])[1] < steps
            steps = size(trajs[i])[1]
        end
    end
    timestamps = 1:round(Int, steps / divider):steps

    record(fig, "$filename.mp4", timestamps; framerate=30) do i
        title[] = L"%$tit $t = %$(timer[i]) $"
        for j in 1:size(trajs)[1]
            patching = size(trajs[j][1][1, :])[1] - size(trajs[j][i][1, :])[1]
            xs[j][] = vcat(trajs[j][i][1, :], (2 * xmax) .* ones(patching))
            ys[j][] = vcat(trajs[j][i][2, :], (2 * ymax) .* ones(patching))
        end
    end
end

colors = [:red, :blue, :green];
markers = [:x, :circle, :circle];

# Maxey Streamline Function Field

In [ ]:
function laplacianField(coord)
    @views x = coord[1, :]
    @views y = coord[2, :]
    return @fastmath @. a * U0 * sin(x / a) * sin(y / a)
end

function vFluid(coord)
    @views x = coord[1, :]
    @views y = coord[2, :]
    @fastmath u = @. U0 * sin(x / a) * cos(y / a)
    @fastmath v = @. -U0 * cos(x / a) * sin(y / a)

    U = hcat(u, v)'
    return U
end

function gradVFluid!(u_x,u_y,v_x,v_y, coord)
    @views x = coord[1, :]
    @views y = coord[2, :]
    @fastmath u_x[:] = @. (U0 / a) * cos.(x / a) * cos.(y / a)
    @fastmath u_y[:] = @. (-U0 / a) * sin.(x / a) * sin.(y / a)

    @fastmath v_x[:] = @. (U0 / a) * sin.(x / a) * sin.(y / a)
    @fastmath v_y[:] = @. (-U0 / a) * cos.(x / a) * cos.(y / a)

    nothing
end

function laplacianField!(Ψ,coord)
    @views x = coord[1, :]
    @views y = coord[2, :]
    Ψ[:] =@fastmath @. a * U0 * sin(x / a) * sin(y / a)
    nothing
end

function vFluid!(U,coord)
    @views x = coord[1, :]
    @views y = coord[2, :]
    @fastmath u = @. U0 * sin(x / a) * cos(y / a)
    @fastmath v = @. -U0 * cos(x / a) * sin(y / a)

    U[:] = hcat(u, v)'
    nothing
end

function gradVFluid!(gU,coord)
    @views x = coord[1, :]
    @views y = coord[2, :]
    @fastmath u_x = @. (U0 / a) * cos.(x / a) * cos.(y / a)
    @fastmath u_y = @. (-U0 / a) * sin.(x / a) * sin.(y / a)

    @fastmath v_x = @. (U0 / a) * sin.(x / a) * sin.(y / a)
    @fastmath v_y = @. (-U0 / a) * cos.(x / a) * cos.(y / a)

    A = hcat(u_x, u_y)'
    B = hcat(v_x, v_y)'
    gU[:] = cat(A, B, dims=3)
    nothing
end

## Plot Fields

In [ ]:
x = LinRange(xmin, xmax, N)
y = LinRange(ymin, ymax, N)

points = meshgrid(x, y)
points = reshape(points, (N^2, 2))';

In [ ]:
PSI_field = laplacianField(points)
PSI_field = reshape(PSI_field, (N, N))

isoPoints = isoPsi(1, 1e-3, PSI_field, x, y, N);

U = vFluid(points)
u_field = reshape(U[1, :], (N, N))
v_field = reshape(U[2, :], (N, N))

P_field = Pressure(points, vFluid);

In [ ]:
# fig = Plots.contourf(x, y, [P_field, PSI_field, u_field, v_field],
#     levels=20, color=:turbo, layout=(2, 2),
#     title=[L"P_{st}" "Ψ" L"$u$" L"$v$"],
#     size=(1200, 1000))
# display(fig)
# savefig("Maxey/Field.png")

## Fluid Trajectory

In [ ]:
plotFluidMaxey = false
if plotFluidMaxey == true
    x0 = reduce(hcat, isoPoints)
    dt = 1e-3
    t0 = 0
    tend = 2 * (xmax - xmin) / U0
    trajFlu = RungeKuttaArgs(vFluid, t0, tend, dt, x0, periodic, a)
    timer = t0:dt:tend
    plotTrajcecotries([trajFlu], timer, PSI_field, [:blue], [:x], "Fluid Trajectories")
    saveTrajcecotries([trajFlu], timer, PSI_field, [:blue], [:x], "Fluid Trajectories", "Maxey/Fluid_Streamline_Animation")
end

## Particle Trajectory

In [ ]:
@fastmath function dvdtP!(F, X, V)
    vDims = size(X)

    gVF = Array{Float64}(undef, (vDims[1],vDims[2],vDims[1]))
    Vfluid = Matrix{Float64}(undef, vDims)
    undisturbed = Matrix{Float64}(undef, vDims) #zeros(vDims)
    vMass = Matrix{Float64}(undef, vDims) 
    ssDrag = Matrix{Float64}(undef, vDims) #zeros(vDims)
    
    vFluid!(Vfluid, X)

    u_x = Array{Float64}(undef, vDims[2])
    u_y = Array{Float64}(undef, vDims[2])
    v_x = Array{Float64}(undef, vDims[2])
    v_y = Array{Float64}(undef, vDims[2])
    gradVFluid!(u_x,u_y,v_x,v_y,X)
    
    @fastmath lift = @. (1 - ρ / ρ_d) * G

    τν = @. (ρ_d * (d_p^2) / (18 * μ))
    @views @fastmath @inbounds ssDrag[1, :] = @. (1 / τν) * (Vfluid-V)[1, :]
    @views @fastmath @inbounds ssDrag[2, :] = @. (1 / τν) * (Vfluid-V)[2, :]

    @fastmath @inbounds undisturbed[1,:] = @. (ρ / ρ_d) * (V[1,:] * u_x + V[2,:] * u_y)
    @fastmath @inbounds undisturbed[2,:] = @. (ρ / ρ_d) * (V[1,:] * v_x + V[2,:] * v_y)

    @fastmath @inbounds vMass[1,:] = @. 0.5 * (ρ / ρ_d) * (Vfluid[1,:] * u_x + Vfluid[2,:] * u_y)
    @fastmath @inbounds vMass[2,:] = @. 0.5 * (ρ / ρ_d) * (Vfluid[1,:] * v_x + Vfluid[2,:] * v_y)

    F[:] = @. dt * (undisturbed + vMass + lift + ssDrag) / (1 + 0.5 * ρ / ρ_d) # + brown
    nothing
end

In [ ]:
# μ = 1.813e-5
global μ = 1e-3
global ρ = 1

global ρ_d = 1000
global d_p = invStokes(0.5)
global V_d = (π / 6) * (d_p^3)
global m_P = V_d * ρ_d
global m_F = V_d * ρ

St = round(Stokes(d_p),sigdigits = 3)
println("Stokes Number is $St\ndiameter: $d_p\ndensity: $ρ_d")
Rmaxie = round((ρ / (0.5 * ρ + ρ_d)),sigdigits=3)
println("R is $Rmaxie")

### Particle Grid

In [ ]:
p_num = round(Int, 30);
de = 0.5;
particleXs = LinRange(xmin + de, xmax - de, p_num);
particleYs = LinRange(ymin + de, ymax - de, p_num);
points = meshgrid(particleXs, particleYs);
points = reshape(points, (p_num^2, 2))';
V0 = vFluid(points);

### Integrate

In [ ]:
dt = 9e-3;
t0 = 0.0;
tend = 20 * (xmax - xmin) / U0
timer = t0:dt:tend;
println("Simulation from t0=$t0 to tend=$tend with dt=$dt")
println("Steps: $(round(Int,(tend-t0)/dt))")

In [ ]:
trajFlu = RungeKutta(vFluid, t0, tend, dt, points, periodic);

In [ ]:
traj = RK4systems(dxdtP!, dvdtP!, t0, tend, dt, points, V0, periodic);

### Animate

In [ ]:
plotTrajcecotries([traj, trajFlu], timer, PSI_field, colors, markers, "Particle Trajectories (Stokes= $(round(St,sigdigits =3)), R=$(round(Rmaxie,sigdigits =3))) "
    , 150)

### Save Vid

In [ ]:
saveTrajcecotries([traj, trajFlu], timer, PSI_field, colors, markers, "Particle Trajectories (Stokes= $(round(St,sigdigits =3)), R=$(round(Rmaxie,sigdigits =3))) ",
    "Maxey/Partictles_R_$(round(Rmaxie,sigdigits =3))_Stokes_$(round(St,sigdigits =3))", 150)